# Tutorial for relaxation calculations using VASP.
## Introduction

Welcome to the MDANSE2024 School tutorial section on using VASP and Phonopy to calculate phonon modes.

This first tutorial will guide you through a simple case of a silicon (Si) crystal.

You will learn how to write and read input/output files from VASP using the Atomic Simulation Environment (ASE). For more information, visit [ASE's official documentation](https://wiki.fysik.dtu.dk/ase/index.html).

Then, you will run VASP on the ILL high-performance computing (HPC) cluster called `masterp`, using a job submission script.

If not already done, please copy this Jupyter Notebook file (.ipynb) and the job submission script `ILL_hpc_vasp_job_local.sh` to your home directory.

Let us start with importing all the necessary Python modules. We need both the numerical libraries and ASE package.
Execute the following cell by selecting it and clicking the $ \blacktriangleright $ symbol in the toolbar.

In [ ]:
import os
import matplotlib.pyplot as plt
from ase.build import bulk
from ase.visualize import view
from ase.calculators.vasp import Vasp

%env VASP_PP_PATH=.

## Example 1: Si crystal

### 1. Preparing the structure

First, we will create a model of silicon crystal, display it and perform simple geometry optimization using VASP.

ASE allows us to quickly generate structures - both molecular and periodic.

Here, we just tell ASE to create a silicon lattice for us.

In [ ]:
si_conventional = bulk('Si', 'fcc', a=5.43, cubic=True)
si_primitive = bulk('Si', 'fcc', a=5.43)

We can visualize the cell using one of several tools which come with ASE

In [ ]:
view(si_conventional, viewer='x3d')

You can rotate the structure and zoom in and out using your mouse.

We can also see details of the generated structure. Let's check unit cell lengths in Angstrom.

In [ ]:
si_conventional.get_cell()

Fractional atomic positions within the cell can be queried with `get_positions()` method. We have 4 Si atoms per conventional cell, so we will get 4 sets of coordinates.

In [ ]:
si_conventional.get_positions()

Similar check can be done for the primitive cell.

In [ ]:
view(si_primitive, viewer='x3d')

In [ ]:
si_primitive.get_cell()

In [ ]:
si_primitive.get_positions()

A more sophisticated optional viewing engine is NGLView.

In [ ]:
try:
    import nglview
except ImportError:
    !pip install nglview
    import nglview
view(si_conventional, viewer='ngl')

### 2. Relaxing the structure (geometry optimization)

Phonon calculations need to be performed on a fully optimized structure, i.e. forces on atoms should be as close to zero as possible. Geometry optimization determines location of atoms in a crystal (or in a molecule) at 0K. The accuracy of the relaxation needs to be high in order to obtain reliable phonon frequencies.

Here, we will use ASE to construct the VASP input files for relaxation. First, create a directory where job files will be kept. This will help in maintaining clean project space between runs.

In [ ]:
os.makedirs('relaxation_si', exist_ok=True)
relaxation_dir='relaxation_si/'

In order to properly define the VASP job, we need to pass a number of parameters describing how we want to run the calculation.

The keyword `PREC` tells VASP how to set default values for various convergence parameters (energy, FFT grid, etc.). Setting this parameter to `Accurate` means VASP is going to help achieving accurate forces and then more accurate phonon spectrum.

The kinetic energy cutoff specifies the energy cutoff for the plane-wave basis set in eV. Here, we will use `300` eV.

The SCF loop will be considered converged, if the difference between current and previous energy will be less than `EDIFF` eV. Here, we choose 5.0e-7 eV per atom.

We also want to achieve convergence for atomic forces - they should be less than 1e-5 eV/A. The negative sign of `EDIFFG` parameter means that the relaxation is stopped when the norms of all the forces are smaller than |ediffg|, which is a more convenient setting.

We will use the standard `PBE` functional, well suited for first order properties like geometries.

Specifying `IBRION` and `ISIF` lets us tweak the relaxation algorithm - we want to perform a relaxation calculation (`IBRION!=0`) and vary atomic positions and cell volume at each step (`ISIF=3`).

For the k-point grid, we want to use a large number of points in all directions. Hence, we chose `8,8,8`. The grid should be centered on the Γ-point.

Lastly, we should not store very large, unneeded files after the job completes, so we need to tell VASP to delete those.


In [ ]:
calc=Vasp(
    directory=relaxation_dir,      
    label='Si_relaxation',
    # Convergence parameters
    prec='Accurate',# Vasp precision tag
    encut=300,      # energy cutoff for the plane-wave basis set in eV
    ediff=5e-7,     # Relax electronic structure until 5e-7 eV energy change between 2 electronic loop
    ediffg=-1e-5,   # Relax ionic forces below 1e-5 eV/A
    # Job settings
    xc="pbe",       # Use the PBE exchange and correlation functional
    ibrion=1,       # Ionic relaxation
    isif=3,         # Relax positions and cell volume
    # K-point grid settings
    kpts=[8,8,8],   # Kpoint grid
    gamma=True,     # Gamma-centered grid
    # Output settings
    lwave=False,    # Do not save wave fuction files (large files)
)

Now, that the VASP job is generated, we can save the required files to the target directory, created earlier.

In [ ]:
calc.write_input(si_conventional)

Now you can examine the content of `relaxation_si` subdirectory.

In [ ]:
import glob
print("\n".join(glob.glob("relaxation_si/*")))

The four files with names in capital letters are VASP input files.

`INCAR` lists parameters used in the energy and force calculations.
`KPOINTS` describes the kpoint grid used.
`POSCAR` describes the unit cell, atom types and their positions.
`POTCAR` contains pseudopotentials for each atomic species.

All four files are required for the calculation we are about to run.


Now it is time to run the VASP calculation. For this, we need to submit the job to the cluster:<br>

a) Connect to the cluster login node: open a terminal and log in with: `$ ssh <your_username>@masterp.ill.fr`.<br>

b) Check that the queue you want to submit to has nodes availables using the `qload` command. If no nodes are available, change the queue in the submit script `vasp.sh`.

c) Copy your job directory `relaxation_si` to the common folder, accessible from the cluster. This folder is called `MyHome` and is linked to directly in your VISA home directory. Use `$cp -r ~/relaxation_si MyHome`

d) Copy your job directory `relaxation_si` to the cluster. Your VISA home folder is mounted on the cluster login node, and it is called as your username. However, the compute nodes don't have access to your home folder, so **you must copy your job directory to the cluster**.<br>
From the cluster login node, use `$ cp -r /nethome/notill/<your_username_initial>/<your_username>/relaxation_si ~<your_username>`.<br>

E.g. for `John Smith`, this would read `$ cp -r /nethome/notill/s/smith/relaxation_si ~/smith`.<br>

e) Launch the job. From the job directory on the cluster (you can move to your job directory using `$ cd relaxation_si`) enter `$ qsub ILL_hpc_vasp_job_local.sh`. You can then monitor your job with `$ qstat`.

f) The calculation should take aprrox. 1 min on 1x8 cpus. Copy back the job folder to your home directory using
`$ cp -rf relaxation_si/* /nethome/notill/<your_username_initial>/<your_username>`.


- Have a look at the results! The main output file of VASP is called `OUTCAR`. You can have a look and check out the forces on silicon atoms during the calculation. Is there anything interesting about the forces?

### Choosing sufficient plane-wave energy cutoff

Choosing an appropriate energy cutoff is critical for obtaining accurate results with plane-wave codes like VASP.
The energy cutoff determines the number of plane waves included in the basis set; higher cutoffs include more plane waves, leading to more accurate but computationally more intensive calculations.

The value of the energy cutoff used is already converged. You may, however, try to assure yourself that `300 eV` is indeed the correct value, by running several calculations with differing cutoffs and comparing the total energy.

(The truncation of the basis set at a finite cutoff energy will lead to an error in the computed total energy and its derivatives. It is possible to reduce the magnitude of the error in a systematic way by increasing the value of the cutoff energy. In principle, the cutoff energy should be increased until the calculated total energy converges within the required tolerance.)

To modify the cutoff, edit the `INCAR` file and change the line
```
ENCUT = 300.000000
```

to another value, then rerun the VASP optimization job and note the total energy. Running calculations between 50 and 500 eV should provide good data to assure convergence of the basis set.


### Exercise 1

Make a copy of the `relaxation_si` directory and name it, say `relaxation_50`.
Change the value of the cutoff parameter and run the calculation
Check the total energy displayed at the end of the `OUTCAR` file and note it.

Repeat for a sequence of cutoffs between 50 and 600 eV.

You can also inspect the `POTCAR` file and look for the `ENMAX` parameter. This value is the maximum recommended energy cutoff for the pseudopotential (or PAW potential) provided for silicon.

| Cutoff (eV)  | Total energy |
| --- | --- |
|50   |  |
|100  |  |
|200  |  |
|250  | |
|300  |  |
|400  |  |
|500  |  |
|600  |  |

In [ ]:
cutoff = [ ... ]
energy = [ ...]

fig, ax = plt.subplots()
ax.plot(cutoff, energy, 'o-')

ax.set(xlabel='Plane wave Cutoff', ylabel='Total Energy',
       title='Convergence test')
ax.grid()

plt.show()

If you compare initial and final cell vector lenghts, you will notice that they did not indeed change.
The initial structure was already optimized, as you can see in the OUTCOR file in the TOTAL-FORCE section.

```
POSITION                                       TOTAL-FORCE (eV/Angst)
 -----------------------------------------------------------------------------------
      0.00000      0.00000      0.00000         0.000000      0.000000      0.000000

```



### Exercise 2

#### Choosing sufficient k-point set:

Similar convergence analysis could be done for the k-points set. The k-point mesh must be sufficiently dense to ensure that the forces (and thus phonon frequencies) are well-converged. For silicon's conventional cell (which is cubic with 8 atoms), a good starting point is a 6×6×6 Monkhorst-Pack k-point mesh.

We could perform convergence tests by optimizing geometries or calculating phobon frequencies at high-symmetry points using different k-point meshes (e.g., 1x1x1, 2x2x2, 4×4×4, 6×6×6, 8×8×8, etc.) to determine the minimum mesh that provides converged results.

To modify the k-points, you need to edit the KPOINTS file in the job directory and replace `8 8 8` with the mesh size requred.

Similarly to Exercise 1, prepare jobs for varying k-point sets and run them on the cluster.
Write down total energies and try to determine if 8x8x8 is a good value.

| k-point set  | Total energy |
| --- | --- |
|1x1x1   |  |
|2x2x2  |  |
|4x4x4  |  |
|6x6x6  |  |
|8x8x8  |  |
|10x10x10  |  |
|12x12x12 |   |

#### Optional questions

Are there any other parameters you could vary?

This is the end of Tutorial 1. Please continue with Tutorial 2 - Geometry Relaxation and Phonon Spectrum of BaTiO3H.